# 第三方工具调用
在ChatGLM3中，模型可以自行调用工具作为辅助来完成任务。在智能客服场景中，工具调用让可以让大模型完成原本无法完成的任务，订单查询任务便是其中的代表。订单查询需要用户个性化的数据，这些数据并没有包含在大模型的训练过程中，因此大模型无法完成查询任务。这时，可以借助第三方工具得到订单数据并传给大模型，这样大模型就拥有了订单查询的能力。

在ChatGLM3调用工具的过程中，我们需要按照官方要求编写一份工具模板，并传给大模型。下面来看一个示例，定义一个名为get_order_detail的工具模板，并指明必须包含参数order_id。接着，通过历史对话的形式将工具模板传给大模型，告诉大模型在解决问题的时候可以利用这个工具模板。之后，我们尝试查询订单，模型会选择合适的工具模板来解析用户的问题并返回需要调用的工具函数方法名以及包含的参数。大模型输出的内容表示需要调用get_order_detail()方法，且参数order_id的值为10002051。get_order_detail工具模板描述代码如下：


首先，加载模型和分词器。

In [2]:
# 加载模型
from transformers import AutoTokenizer, AutoModel
model_path="chatglm3-6b"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).quantize(4).cuda()
model = model.eval()

/home/whwang22/.conda/envs/book_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.19s/it]


In [3]:
tools=[
    {
        "name": "get_order_detail",
        "description": "查询订单的详细信息。",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "description": "需要查询的订单编号"
                }
            },
            "required": ['order_id']
        }
    }]
system_info = {"role": "system", 
               "content": "Answer the following questions as best as you can. You have access to the following tools:", 
               "tools": tools}
history=[system_info]
query="查询编号为10002051的订单"
# 第一次调用模型
response,history=model.chat(tokenizer,query,history=history) 
response

{'name': 'get_order_detail', 'parameters': {'order_id': 10002051}}

到目前为止是第一次调用大模型，大模型返回了需要调用的方法以及传入的参数，但是我们并没有定义get_order_detail()这个方法，接下来我们需要实现它。获取订单详情的实现方法的示例如下，读者可根据实际的业务场景进行不同的实现。

In [4]:
# 定义新的工具实现的方法
from typing import Annotated
import pandas as pd
def get_order_detail(
    order_id:Annotated[str,'The order number to be queried', True]
) -> str:
    """
    Get the detail for `order_id`
    """
    order_id=int(order_id)
    # 加载订单数据源
    order_data=pd.read_excel('订单数据.xlsx')
    # 获取存在的订单编号
    order_id_list=order_data['商品编号'].to_list()
    # 判断输入的订单编号是否存在
    if order_id in order_id_list:
        # 获取订单编号的索引
        idx=order_id_list.index(order_id)
        order_detail=eval(order_data.loc[idx].to_json(force_ascii=False))
        # 时间戳格式在to_json后会被破坏，需要重新设置时间
        order_detail['下单时间']=order_data.loc[idx]['下单时间'].strftime("%Y-%m-%d %H:%M:%S")
        # 如果订单状态是已取消，则不需要提供支付单号和支付时间
        if order_detail['订单状态'] == '已取消':
            del order_detail['支付单号']
            del order_detail['支付时间']
        else:
            order_detail['支付时间']=order_data.loc[idx]['支付时间'].strftime("%Y-%m-%d %H:%M:%S")
        return str(order_detail)
    # 如果订单编号不存在,则不做查询
    else:
        return '查询的订单不存在'

完成方法的实现后，需要使用官方提供的方法注册工具，接着利用工具获取订单的详情信息传给大模型，让其完成最终任务。以下所示的代码中，register_tool方法和dispatch_tool方法都是ChatGLM3官方提供的方法，register_tool用来注册自定义的工具，dispatch_tool用来调用工具，将得到的结果输入大模型完成任务，需要注意的是，根据官方规定工具获得值需要在role参数中指明，否则大模型会混淆。

In [5]:
from tool_registry import register_tool,dispatch_tool
register_tool(get_order_detail)
res=dispatch_tool(response['name'],response['parameters'])
# 这里 role="observation" 表示输入的是工具调用的返回值而不是用户输入，不能省略。
response,history=model.chat(tokenizer,res,role='observation',history=history)
response

[registered tool] {'description': 'Get the detail for `order_id`',
 'name': 'get_order_detail',
 'params': [{'description': 'The order number to be queried',
             'name': 'order_id',
             'required': True,
             'type': 'str'}]}


"您好，根据您的查询，我已经成功获取到编号为10002051的订单的详细信息。该商品为[追加限量]GSC 食戟之灵 薙切绘里奈 手办 再版，价格为102.75元。下单时间为2019年2月9日16:00:00，支付时间为2019年2月9日16:00:40，支付单号为'4083290968857591808'。该订单状态为待支付，是否为海外购为否。请问还有其他需要帮助的地方吗？"

我们以历史对话的形式将第三方订单传给大模型，让大模型在解决问题的时候可以利用这个工具模板。实现订单查询，并输出查询结果。